In [1]:
import pandas as pd
df = pd.read_csv('amazon_objets_connectes_v27.csv')
df.head()

ModuleNotFoundError: No module named 'pandas'

In [1]:
import pandas as pd
df = pd.read_csv('amazon_objets_connectes_v27.csv')
df.head()

,title,brand,price,rating,votes,sales_last_month,image_url,category,rank,scraped_at
0,Garmin – Forerunner 55 - Montre GPS multi-acti...,Garmin,138.99,4.6,6532.0,1.0,https://m.media-amazon.com/images/I/61eTqEILa9...,Objets connectés,1,2025-06-24 11:06:24.687706
1,Garmin Forerunner 255 - Montre GPS Multisports...,Garmin,203.99,4.5,1690.0,700.0,https://m.media-amazon.com/images/I/71tjy7Umf0...,Objets connectés,2,2025-06-24 11:06:24.688216
2,"Garmin Forerunner 255, Music - Montre GPS Mult...",Garmin,244.00,4.5,616.0,700.0,https://m.media-amazon.com/images/I/51BxNNStky...,Objets connectés,3,2025-06-24 11:06:24.688871
3,"Insta360 X5 - caméra d'action 360° 8K étanche,...",Insta360,589.99,4.5,557.0,300.0,https://m.media-amazon.com/images/I/71YjsbkkO9...,Objets connectés,4,2025-06-24 11:06:24.689625
4,Garmin Forerunner 255 – GPS-Laufuhr mit indivi...,Garmin,203.99,4.6,662.0,400.0,https://m.media-amazon.com/images/I/51oM3U58x2...,Objets connectés,5,2025-06-24 11:06:24.690255


In [4]:
df['sales_last_month'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2280 entries, 0 to 2279
Series name: sales_last_month
Non-Null Count  Dtype  
--------------  -----  
142 non-null    float64
dtypes: float64(1)
memory usage: 17.9 KB


In [ ]:
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

# Configuration Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

base_url = "https://www.amazon.fr/s?i=electronics&srs=4551203031&rh=n%3A4551203031&s=popularity-rank&fs=true&page={}"

products = []
page = 1
max_pages = 150

while page <= max_pages:
    print(f"Scraping page {page}")
    driver.get(base_url.format(page))
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("div", {"data-component-type": "s-search-result"})

    for rank, item in enumerate(items, start=1 + (page - 1) * len(items)):
        title_elem = item.h2
        title = title_elem.text.strip() if title_elem else None

        price = None
        price_whole = item.select_one('span.a-price > span.a-offscreen')
        if price_whole:
            price = price_whole.text.strip().replace("€", "").replace(",", ".")
            try:
                price = float(price)
            except:
                price = None

        rating = None
        rating_tag = item.select_one("span.a-icon-alt")
        if rating_tag:
            rating_match = re.search(r"(\d+,\d+)", rating_tag.text)
            if rating_match:
                rating = float(rating_match.group(1).replace(",", "."))

        # Bloc votes
        votes = None
        votes_text = None
        votes_elem = item.find("span", class_="a-size-base s-underline-text")
        if votes_elem:
            votes_text = votes_elem.text.strip()
        else:
            alt_votes = item.select_one("div.a-row.a-size-small span.a-size-base")
            if alt_votes:
                votes_text = alt_votes.text.strip()

        if votes_text:
            cleaned = re.sub(r"[^\d]", "", votes_text)
            if cleaned.isdigit():
                votes = int(cleaned)

        # Bloc ventes - méthode 1
        sales = None
        sales_elem = item.find("span", class_="a-size-base a-color-secondary")
        if sales_elem and "acheté" in sales_elem.text:
            sales_match = re.search(r"(\d[\d\s]+)", sales_elem.text)
            if sales_match:
                sales = int(sales_match.group(1).replace(" ", "").replace("\u202f", ""))

        # Bloc ventes - fallback via page produit
        link_elem = item.find("a", class_="a-link-normal s-no-outline")
        product_url = f"https://www.amazon.fr{link_elem['href']}" if link_elem else None

        if sales is None and product_url:
            try:
                driver.get(product_url)
                time.sleep(2)
                detail_soup = BeautifulSoup(driver.page_source, "html.parser")
                detail_text = detail_soup.get_text()
                detail_match = re.search(r"(\d[\d\s]+)\s+achetés? au cours du dernier mois", detail_text)
                if detail_match:
                    sales = int(detail_match.group(1).replace(" ", "").replace("\u202f", ""))
            except Exception as e:
                print(f"❌ Erreur dans la page produit : {product_url}\n{e}")

        image_elem = item.find("img")
        image_url = image_elem['src'] if image_elem else None

        brand = None
        if title:
            brand = title.split()[0]

        products.append({
            "title": title,
            "brand": brand,
            "price": price,
            "rating": rating,
            "votes": votes,
            "sales_last_month": sales,
            "image_url": image_url,
            "category": "Objets connectés",
            "rank": rank,
            "scraped_at": datetime.now()
        })

    page += 1

driver.quit()

dfv35 = pd.DataFrame(products)
dfv35.to_csv("amazon_objets_connectes_v35.csv", index=False)
print("✅ Données sauvegardées : amazon_objets_connectes_v35.csv")


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
✅ Données sauvegardées : amazon_objets_connectes_v35.csv


In [9]:
dfv35 = pd.read_csv("amazon_objets_connectes_v35.csv")
dfv35.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             216 non-null    object 
 1   brand             216 non-null    object 
 2   price             213 non-null    float64
 3   rating            211 non-null    float64
 4   votes             211 non-null    float64
 5   sales_last_month  126 non-null    float64
 6   image_url         216 non-null    object 
 7   category          216 non-null    object 
 8   rank              216 non-null    int64  
 9   scraped_at        216 non-null    object 
dtypes: float64(4), int64(1), object(5)
memory usage: 17.0+ KB


In [10]:
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

# Configuration Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

base_url = "https://www.amazon.fr/s?i=electronics&srs=4551203031&rh=n%3A4551203031&s=popularity-rank&fs=true&page={}"

products = []
page = 1
max_pages = 10

while page <= max_pages:
    print(f"🔍 Scraping page {page}")
    driver.get(base_url.format(page))
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("div", {"data-component-type": "s-search-result"})

    for rank, item in enumerate(items, start=1 + (page - 1) * len(items)):
        title_elem = item.h2
        title = title_elem.text.strip() if title_elem else None

        brand = title.split()[0] if title else None

        price_elem = item.select_one('span.a-price > span.a-offscreen')
        price = None
        if price_elem:
            price = price_elem.text.strip().replace("€", "").replace(",", ".")
            try:
                price = float(price)
            except:
                price = None

        rating = None
        rating_tag = item.select_one("span.a-icon-alt")
        if rating_tag:
            match = re.search(r"(\d+,\d+)", rating_tag.text)
            if match:
                rating = float(match.group(1).replace(",", "."))

        # Votes
        votes = None
        votes_text = None
        votes_elem = item.find("span", class_="a-size-base s-underline-text")
        if not votes_elem:
            alt_votes = item.select_one("div.a-row.a-size-small span.a-size-base")
            if alt_votes:
                votes_text = alt_votes.text.strip()
        else:
            votes_text = votes_elem.text.strip()

        if votes_text:
            cleaned = re.sub(r"[^\d]", "", votes_text)
            if cleaned.isdigit():
                votes = int(cleaned)

        # Ventes dernier mois
        sales = None
        sales_elem = item.find("span", class_="a-size-base a-color-secondary")
        if sales_elem and "acheté" in sales_elem.text:
            match = re.search(r"(\d[\d\s]+)", sales_elem.text)
            if match:
                sales = int(match.group(1).replace(" ", "").replace("\u202f", ""))

        # Image
        image_elem = item.find("img")
        image_url = image_elem['src'] if image_elem else None

        # URL produit
        link_elem = item.h2.find("a") if item.h2 else None
        url = "https://www.amazon.fr" + link_elem['href'] if link_elem else None

        # Badge Prime
        prime = bool(item.select_one("i.a-icon-prime"))

        # Enregistrement
        products.append({
            "title": title,
            "brand": brand,
            "price": price,
            "rating": rating,
            "votes": votes,
            "sales_last_month": sales,
            "image_url": image_url,
            "url": url,
            "prime": prime,
            "category": "Objets connectés",
            "rank": rank,
            "scraped_at": datetime.now()
        })

    page += 1

driver.quit()

dfv36 = pd.DataFrame(products)
dfv36.to_csv("amazon_objets_connectes_v36.csv", index=False)
print("✅ Données sauvegardées : amazon_objets_connectes_v36.csv")


🔍 Scraping page 1
🔍 Scraping page 2
🔍 Scraping page 3
🔍 Scraping page 4
🔍 Scraping page 5
🔍 Scraping page 6
🔍 Scraping page 7
🔍 Scraping page 8
🔍 Scraping page 9
🔍 Scraping page 10
✅ Données sauvegardées : amazon_objets_connectes_v36.csv


In [11]:
dfv36 = pd.read_csv("amazon_objets_connectes_v36.csv")
dfv36.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             240 non-null    object 
 1   brand             240 non-null    object 
 2   price             235 non-null    float64
 3   rating            234 non-null    float64
 4   votes             234 non-null    float64
 5   sales_last_month  137 non-null    float64
 6   image_url         240 non-null    object 
 7   url               0 non-null      float64
 8   prime             240 non-null    bool   
 9   category          240 non-null    object 
 10  rank              240 non-null    int64  
 11  scraped_at        240 non-null    object 
dtypes: bool(1), float64(5), int64(1), object(5)
memory usage: 21.0+ KB


In [ ]:
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

# Configuration Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

base_url = "https://www.amazon.fr/s?i=electronics&srs=4551203031&rh=n%3A4551203031&s=popularity-rank&fs=true&page={}"

products = []
page = 1
max_pages = 10  # ajuste selon ton besoin

while page <= max_pages:
    print(f"Scraping page {page}")
    driver.get(base_url.format(page))
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("div", {"data-component-type": "s-search-result"})

    for rank, item in enumerate(items, start=1 + (page - 1) * len(items)):
        title_elem = item.h2
        title = title_elem.text.strip() if title_elem else None

        # URL produit
        url = None
        link_elem = item.select_one("h2 a.a-link-normal")
        if link_elem and link_elem.has_attr("href"):
            url = "https://www.amazon.fr" + link_elem['href']

        # Prix
        price = None
        price_whole = item.select_one('span.a-price > span.a-offscreen')
        if price_whole:
            try:
                price = float(price_whole.text.strip().replace("€", "").replace(",", "."))
            except:
                price = None

        # Note
        rating = None
        rating_tag = item.select_one("span.a-icon-alt")
        if rating_tag:
            rating_match = re.search(r"(\d+,\d+)", rating_tag.text)
            if rating_match:
                rating = float(rating_match.group(1).replace(",", "."))

        # Votes
        votes = None
        votes_text = None
        votes_elem = item.find("span", class_="a-size-base s-underline-text")
        if votes_elem:
            votes_text = votes_elem.text.strip()
        else:
            alt_votes = item.select_one("div.a-row.a-size-small span.a-size-base")
            if alt_votes:
                votes_text = alt_votes.text.strip()

        if votes_text:
            cleaned = re.sub(r"[^\d]", "", votes_text)
            if cleaned.isdigit():
                votes = int(cleaned)

        # Ventes dernier mois
        sales = None
        sales_elem = item.find("span", class_="a-size-base a-color-secondary")
        if sales_elem and "acheté" in sales_elem.text:
            sales_match = re.search(r"(\d[\d\s]+)", sales_elem.text)
            if sales_match:
                sales = int(sales_match.group(1).replace(" ", "").replace("\u202f", ""))

        # Image
        image_elem = item.find("img")
        image_url = image_elem['src'] if image_elem else None

        # Marque
        brand = title.split()[0] if title else None

        # Prime
        prime = bool(item.select_one("i.a-icon-prime"))

        products.append({
            "title": title,
            "brand": brand,
            "price": price,
            "rating": rating,
            "votes": votes,
            "sales_last_month": sales,
            "image_url": image_url,
            "url": url,
            "prime": prime,
            "category": "Objets connectés",
            "rank": rank,
            "scraped_at": datetime.now()
        })

    page += 1

driver.quit()

# Sauvegarde
dfv37 = pd.DataFrame(products)
dfv37.to_csv("amazon_objets_connectes_v37.csv", index=False)
print("✅ Données sauvegardées : amazon_objets_connectes_v37.csv")


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
✅ Données sauvegardées : amazon_objets_connectes_v37.csv


In [13]:
dfv37 = pd.read_csv("amazon_objets_connectes_v37.csv")
dfv37.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             240 non-null    object 
 1   brand             240 non-null    object 
 2   price             235 non-null    float64
 3   rating            234 non-null    float64
 4   votes             234 non-null    float64
 5   sales_last_month  138 non-null    float64
 6   image_url         240 non-null    object 
 7   url               0 non-null      float64
 8   prime             240 non-null    bool   
 9   category          240 non-null    object 
 10  rank              240 non-null    int64  
 11  scraped_at        240 non-null    object 
dtypes: bool(1), float64(5), int64(1), object(5)
memory usage: 21.0+ KB


In [18]:
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

# Configuration Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

base_url = "https://www.amazon.fr/s?i=electronics&srs=4551203031&rh=n%3A4551203031&s=popularity-rank&fs=true&page={}"

products = []
page = 1
max_pages = 2000  # Tu peux augmenter ce chiffre

while page <= max_pages:
    print(f"Scraping page {page}")
    driver.get(base_url.format(page))
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("div", {"data-component-type": "s-search-result"})

    for rank, item in enumerate(items, start=1 + (page - 1) * len(items)):
        title_elem = item.h2
        title = title_elem.text.strip() if title_elem else None

        price = None
        price_whole = item.select_one('span.a-price > span.a-offscreen')
        if price_whole:
            try:
                price = float(price_whole.text.strip().replace("€", "").replace(",", "."))
            except:
                price = None

        rating = None
        rating_tag = item.select_one("span.a-icon-alt")
        if rating_tag:
            rating_match = re.search(r"(\d+,\d+)", rating_tag.text)
            if rating_match:
                rating = float(rating_match.group(1).replace(",", "."))

        # Votes (avis clients)
        votes = None
        votes_text = None
        votes_elem = item.find("span", class_="a-size-base s-underline-text")
        if votes_elem:
            votes_text = votes_elem.text.strip()
        else:
            alt_votes = item.select_one("div.a-row.a-size-small span.a-size-base")
            if alt_votes:
                votes_text = alt_votes.text.strip()
        if votes_text:
            cleaned = re.sub(r"[^\d]", "", votes_text)
            if cleaned.isdigit():
                votes = int(cleaned)

        # Ventes le mois dernier
        sales = None
        sales_elem = item.find("span", class_="a-size-base a-color-secondary")
        if sales_elem and "acheté" in sales_elem.text:
            sales_match = re.search(r"(\d[\d\s]+)", sales_elem.text)
            if sales_match:
                sales = int(sales_match.group(1).replace(" ", "").replace("\u202f", ""))

        # Image
        image_elem = item.find("img")
        image_url = image_elem['src'] if image_elem else None

        # Marque
        brand = None
        if title:
            brand = title.split()[0]

        # URL produit
        url = None
        link_tag = item.select_one("a.a-link-normal.s-link-style.a-text-normal")
        if link_tag and link_tag.get("href"):
            url = "https://www.amazon.fr" + link_tag["href"]

        # Prime (booléen)
        prime = bool(item.select_one("i.a-icon-prime"))

        products.append({
            "title": title,
            "brand": brand,
            "price": price,
            "rating": rating,
            "votes": votes,
            "sales_last_month": sales,
            "image_url": image_url,
            "url": url,
            "prime": prime,
            "category": "Objets connectés",
            "rank": rank,
            "scraped_at": datetime.now()
        })

    page += 1

driver.quit()

# Sauvegarde
dfv38 = pd.DataFrame(products)
dfv38.to_csv("amazon_objets_connectes_v38.csv", index=False)
print("✅ Données sauvegardées : amazon_objets_connectes_v38.csv")


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50
Scraping page 51
Scraping page 52
Scraping page 53
Scraping page 54
Scraping page 55
Scraping page 56
Scraping page 57
Scraping page 58
Scraping page 59
Scrapi

In [19]:
dfv38 = pd.read_csv("amazon_objets_connectes_v38.csv")
dfv38.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47904 entries, 0 to 47903
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             47904 non-null  object 
 1   brand             47904 non-null  object 
 2   price             47152 non-null  float64
 3   rating            5259 non-null   float64
 4   votes             5259 non-null   float64
 5   sales_last_month  142 non-null    float64
 6   image_url         47904 non-null  object 
 7   url               47904 non-null  object 
 8   prime             47904 non-null  bool   
 9   category          47904 non-null  object 
 10  rank              47904 non-null  int64  
 11  scraped_at        47904 non-null  object 
dtypes: bool(1), float64(4), int64(1), object(6)
memory usage: 4.1+ MB


In [ ]:
#!!!!!! pip install pyarrow !!!!!!!!!


from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

# Configuration Selenium
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

base_url = "https://www.amazon.fr/s?i=electronics&srs=4551203031&rh=n%3A4551203031&s=popularity-rank&fs=true&page={}"

products = []
page = 1
max_pages = 190  # Nombre de pages à scraper (ajuster si nécessaire)

while page <= max_pages:
    print(f"Scraping page {page}")
    driver.get(base_url.format(page))
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    items = soup.find_all("div", {"data-component-type": "s-search-result"})

    for rank, item in enumerate(items, start=1 + (page - 1) * len(items)):
        title_elem = item.h2
        title = title_elem.text.strip() if title_elem else None

        price = None
        price_whole = item.select_one("span.a-price > span.a-offscreen")
        if price_whole:
            try:
                price = float(
                    price_whole.text.strip().replace("€", "").replace(",", ".")
                )
            except:
                price = None

        rating = None
        rating_tag = item.select_one("span.a-icon-alt")
        if rating_tag:
            rating_match = re.search(r"(\d+,\d+)", rating_tag.text)
            if rating_match:
                rating = float(rating_match.group(1).replace(",", "."))

        # Votes (avis clients)
        votes = None
        votes_text = None
        votes_elem = item.find("span", class_="a-size-base s-underline-text")
        if votes_elem:
            votes_text = votes_elem.text.strip()
        else:
            alt_votes = item.select_one("div.a-row.a-size-small span.a-size-base")
            if alt_votes:
                votes_text = alt_votes.text.strip()
        if votes_text:
            cleaned = re.sub(r"[^\d]", "", votes_text)
            if cleaned.isdigit():
                votes = int(cleaned)

        # Ventes le mois dernier
        sales = None
        sales_elem = item.find("span", class_="a-size-base a-color-secondary")
        if sales_elem and "acheté" in sales_elem.text:
            sales_match = re.search(r"(\d[\d\s]+)", sales_elem.text)
            if sales_match:
                sales = int(sales_match.group(1).replace(" ", "").replace("\u202f", ""))

        # Image
        image_elem = item.find("img")
        image_url = image_elem["src"] if image_elem else None

        # URL produit
        url = None
        link_tag = item.select_one("a.a-link-normal.s-link-style.a-text-normal")
        if link_tag and link_tag.get("href"):
            url = "https://www.amazon.fr" + link_tag["href"]

        # Prime (booléen)
        prime = bool(item.select_one("i.a-icon-prime"))

        products.append(
            {
                "title": title,
                "price": price,
                "rating": rating,
                "votes": votes,
                "sales_last_month": sales,
                "image_url": image_url,
                "url": url,
                "prime": prime,
                "scraped_at": datetime.now(),
            }
        )

    page += 1

driver.quit()

# Sauvegarde
df_listings = pd.DataFrame(products)
df_listings.to_parquet("amazon_objets_connectes_.parquet", index=False)
print("✅ Données sauvegardées : amazon_objets_connectes_.parquet")


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50
Scraping page 51
Scraping page 52
Scraping page 53
Scraping page 54
Scraping page 55
Scraping page 56
Scraping page 57
Scraping page 58
Scraping page 59
Scrapi